In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR

In [ ]:
def replace_nan(df):
    # get a list of all the columns containing NaN
    nan_cols = df[df.columns[df.isnull().any()]].columns
    nan_cols = nan_cols.drop('bikes')
    # compute and fill each NaN with the columns mean
    df[nan_cols] = df[nan_cols].fillna(value=df[nan_cols].mean())

    
def show_nans(df):
    print(np.unique(df['station']))
    print(df.shape[0] - df.dropna().shape[0])
#     print(df[df.columns[df.isnull().any()]].columns)
    print(df.isnull().any())
    print()
    

# converting weekdays into integers [1-7]
def convert_weekdays(df):
    df = df.replace(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    [1, 2, 3, 4, 5, 6, 7], inplace=True)
    
def score_abs_error(model, data, n_docks, round_ = False):
    
    print(data.shape)
    print(n_docks.shape)
    
    if round_ == True:
        y_pred = np.around(  model.predict(data.iloc[:,:-1].to_numpy()) * n_docks  )
    else:
        y_pred = model.predict(data.iloc[:,:-1].to_numpy()) * n_docks
    y_gold = data["bikes"].to_numpy() * n_docks
    
    return mean_absolute_error(y_gold, y_pred)

def reasonable_predictions(model, data):
    y_pred = model.predict(data.to_numpy())
    
    y_pred = np.around(y_pred)
    return y_pred

def load_data():
    df = []
    
    for path in Path('./Train/Train').rglob('*.csv'):
        tmp = pd.read_csv(path)
        df.append(tmp)
    
    df = pd.concat(df, ignore_index=True)
    return clean_data(df)
        
def clean_data(df):
    convert_weekdays(df)
    # These three features do not change value throughout the entire training set
    del df["month"]
    del df["year"]
    del df['precipitation.l.m2']
    df = df.dropna(axis='rows')
    return df



In [ ]:
# Creating the dataset
df = load_data()

## New Features?
Thought we could try making new features... people will use bikes more often when they aren't at work. Made a new feature that combines weekends and bank holidays? Might be more relevent?

In [ ]:
def is_hol_weekend(row):
    if row['weekday'] == 6 or row['weekday'] == 7:
        return 1
    if row['isHoliday'] == 1:
        return 1
    return 0


df['isOff'] = df.apply(is_hol_weekend,axis=1)

In [ ]:
plt.hist(df["isOff"])

In [ ]:
plt.hist(df["bikes"])

In [ ]:
forest = RandomForestRegressor(n_estimators=500, n_jobs=6)
print("initialised")
forest.fit(df.iloc[:,:-1].to_numpy(), df["bikes"].to_numpy())

importances = forest.feature_importances_
imp_indixes = np.argsort(importances)[::-1]
feature_order = df.columns[:-1][imp_indixes]
importances = importances[imp_indixes]

imp_df = pd.DataFrame(data = importances, index = feature_order, columns=["relative_importance"])

# print(score_abs_error(forest, df))

In [ ]:
lowest_ranked_10 = feature_order[-5:]
for feature in lowest_ranked_10:
    del df[feature]
print(imp_df)

In [ ]:
train, val = train_test_split(df, test_size=0.2)


columns = list(df.columns[-6:])
print(columns)
for c in columns:
    train[c] = train[c].to_numpy() / train["numDocks"].to_numpy()
    val[c] = val[c].to_numpy() / val["numDocks"].to_numpy()


train_docks = train["numDocks"]
val_docks = val["numDocks"]
del train["numDocks"]
del val["numDocks"]

val

# See all Rows/Cols

#pd.set_option('display.max_rows', 23)

In [ ]:
df

# Random elimination parameter tuning
## Random forest regressor

This cell uses `HalvingRandomSearchCV` to find near-optimal parameters for a random forest regressor. It takes a while to run with these parameters.

In [ ]:
# from time import time
# from sklearn.experimental import enable_halving_search_cv  # noqa
# from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV
# from scipy.stats import randint


# # forest_boost = GradientBoostingRegressor(n_estimators=n_est, loss='squared_error', learning_rate=0.2, max_depth=2, verbose=1)
# #  forest_boost = SVR()
# searched_boost = RandomForestRegressor()
# searched_boost = GradientBoostingRegressor()



# param_distributions = {"max_depth":  [2,3,4],   #, 5, 6, None],
#                        "min_samples_split": np.around(np.linspace(2,20,10)).astype(np.int32),
#                        "learning_rate": np.linspace(0.0001,1,10),
#                        "n_estimators": np.linspace(5, 5000, 50).astype(np.int32)
#                       }

# search = HalvingRandomSearchCV(searched_boost, param_distributions,
#                                resource='n_samples', aggressive_elimination=True, min_resources = 1000,
#                                factor = 2 ,cv = 2,    # n_candidates =  25
#                                random_state=0, verbose=1, n_jobs=6).fit(train.iloc[:,:-1].to_numpy(), train["bikes"].to_numpy())

# print(search.best_params_)

In [ ]:
from time import time
start = time()
print("initialised")
#forest_boost = GradientBoostingRegressor(**search.best_params_)
forest_boost = GradientBoostingRegressor(n_estimators= 5000, min_samples_split= 10, max_depth= 3, learning_rate= 0.2, verbose = 1)
forest_boost.fit(train.iloc[:,:-1].to_numpy(), train["bikes"].to_numpy())
print(f'fitted in {time() - start}s')

In [ ]:


print(score_abs_error(forest_boost, val, val_docks))
print(score_abs_error(forest_boost, val, val_docks, round_ = True))

In [ ]:


# results = pd.DataFrame(search.cv_results_)
# results["params_str"] = results.params.apply(str)
# params = search.param_distributions
# # results.drop_duplicates(subset=("params_str", "iter"), inplace=True)
# learning_rates = params["learning_rate"]
# mean_scores = results.pivot(
#     index="iter", columns="params_str", values="mean_test_score"
# )

# fig, ax = plt.subplots(figsize=(16,12))
# mean_scores.plot(legend=False, alpha=0.6, ax = ax, linewidth=8)

# labels = [
#     f"iter={i}\nn_samples={search.n_resources_[i]} \nn_candidates={search.n_candidates_[i]}"# \nn_estimators={params["n_estimators"][i]} "
#     for i in range(search.n_iterations_)
# ]

# ax.set_xticks(range(search.n_iterations_))
# ax.set_xticklabels(labels, rotation=45, multialignment="left")
# ax.set_title("Scores of candidates over iterations")
# ax.set_ylabel("mean test score", fontsize=15)
# ax.set_xlabel("iterations", fontsize=15)
# plt.tight_layout()
# plt.show()


In [ ]:
# try:
#     n_est = search.best_params_["n_estimators"]
# except:
n_est = 5000

import matplotlib.pyplot as plt
test_score = np.zeros((n_est,), dtype=np.float64)

y_test = val["bikes"]
#y_pred = reasonable_predictions(forest_boost, val.iloc[:, :-1])

for i, y_pred in enumerate(forest_boost.staged_predict(val.iloc[:,:-1])):
    test_score[i] = mean_absolute_error(y_test* val_docks, y_pred* val_docks)
    #test_score[i] = forest_boost.loss_(y_test, np.around(y_pred))

#test_score = test_score * val_docks

fig = plt.figure(figsize=(12, 12))
plt.subplot(1, 1, 1)
plt.title("Training and validation error")
plt.plot(
    np.arange(n_est) + 1,
    forest_boost.train_score_,
    "b-",
    label="Training Set",
)
plt.plot(
    np.arange(n_est) + 1, test_score, "r-", label="Validation Set"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Squared error")
#plt.yscale('log')
fig.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.hist(y_test - forest_boost.predict(val.iloc[:,:-1]), bins = 30)
plt.show()

In [ ]:
test = pd.read_csv('test.csv')

ids = test["Id"]

del test["Id"]
del test["month"]
del test["year"]

test.dropna()

convert_weekdays(test)
# test[test.columns] = scaler.fit_transform(test[test.columns])

# for feature in lowest_ranked_10:
#     del test[feature]


columns = list(test.columns[-5:])
print(columns)
for c in columns:
    test[c] = test[c].to_numpy() / test["numDocks"].to_numpy()

print(test.head())

plt.hist(test["bikes_3h_ago"])
plt.show()

test_docks = test["numDocks"].to_numpy()
print(test_docks)
del test["numDocks"]


#y_pred = forest_boost.predict(test)

print(np.around(forest_boost.predict(test.to_numpy()) * test_docks))

y_pred = np.around(forest_boost.predict(test.to_numpy()) * test_docks)
print(repr(y_pred))
sub_df = pd.DataFrame(data=y_pred, index = ids, columns = ["bikes"])

sub_df.index.name = 'Id'

print(sub_df.head())
print(sub_df.shape)

sub_df.to_csv("submission.csv")